## install reqs

In [ ]:
!pip install tensorflow seaborn matplotlib scikit-learn


## connect to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
# List the contents of your shared drives
print(os.listdir('/content/drive/MyDrive'))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['VID_20210528_174657_178.mp4', 'VID_20210528_174701_440.mp4', 'Infrastructure VS. AD-HOC.pdf.gdrive', 'افكار لكتابه السيره الذاتية.gdoc', 'British Council Certificate.pdf', 'IMG_20211010_131359 (4).jpg', 'IMG_20211010_131359 (3).jpg', 'IMG_20211010_131359 (2).jpg', 'IMG_20211010_131359 (1).jpg', 'IMG_20211010_131359.jpg', 'IMG_20211010_192217.jpg', 'Screenshot_2021-10-21-11-18-21-80_40deb401b9ffe8e1df2f1cc5ba480b12.jpg', 'exp1s.pdf', 'EXP2.pdf', 'KIRSSOF.pdf', '16372573386196998407231969063766.jpg', '16372575382426078223689767951657.jpg', '555555.pdf', 'abdelrhman Zain Mohamed 2101646(2).pdf', 'EXP 5.pdf', 'EXP 6.pdf', 'Abdelrhman zain mohamed 2101646(3).pdf', 'نسخة من Sections 7 & 8.pdf', 'exp1sssss.pdf', 'abdelrhman zain 1 .pdf', 'ABDELRHman zain 2 pdf.pdf', 'ABDEL rhman zain . 3.pdf', 'exp measure 4 .pdf', 'Introduction.pdf', 'exp 6.pdf', 'exp 7.pdf', 'ex

## load Data and make Data augmentation to increase data

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Teeth_Dataset/Training'
validation_dir = '/content/drive/MyDrive/Teeth_Dataset/Validation'
test_dir = '/content/drive/MyDrive/Teeth_Dataset/Testing'

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.5, 1.0]
)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical'
)

test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)


Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


## load model and fine-tune it

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the base model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the number of samples in the validation generator
print("Number of validation samples:", validation_generator.samples)
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50
)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# Recompile the model for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=50
)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy}')


## Evaluate the model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get true labels and predictions
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = y_pred.argmax(axis=-1)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification report
print(classification_report(y_true, y_pred_classes, target_names=train_generator.class_indices.keys()))
